In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2    
from tqdm import tqdm
import random as random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D,BatchNormalization
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [0]:
imagedetails = []
imagelabels = []
IMG_SIZE=150
hispa_dir='/content/drive/My Drive/DATASET/LabelledRice/Labelled/Hispa'
leafblast_dir='/content/drive/My Drive/DATASET/LabelledRice/Labelled/LeafBlast'
brownspot_dir='/content/drive/My Drive/DATASET/LabelledRice/Labelled/BrownSpot'
Healthy_dir='/content/drive/My Drive/DATASET/LabelledRice/Labelled/Healthy'

In [5]:
def read_image(label,DIR):
    
    for img in tqdm(os.listdir(DIR)):
    #for img in DIR:
        path = os.path.join(DIR,img)
        #checking image type
        _, ftype = os.path.splitext(path)
        if ftype == ".jpg":
       
            image = cv2.imread(path,cv2.IMREAD_COLOR)
            image = cv2.resize(image, (IMG_SIZE,IMG_SIZE),interpolation = cv2.INTER_AREA)
            imagedetails.append(np.array(image))
            imagelabels.append(str(label))

read_image("hispa",hispa_dir)
read_image("leafblast",leafblast_dir)
read_image("brownspot",brownspot_dir)
read_image("healthy",Healthy_dir)

100%|██████████| 1488/1488 [15:06<00:00,  1.64it/s]


In [6]:
#LabelEncoder() is a default function which will be used to convert the strings label to int label
encoder = LabelEncoder()
#converting the images into arrays which will have its intensity values
X = np.array(imagedetails)
print(X)
#divide by 255 is to make the analysis easy because each color 8 plains (2^8==256)
X = X/255
print(X)
print(imagelabels)
#transformig the image labels into integers because it easy to deal with integers compared to strings;
y = encoder.fit_transform(imagelabels)
#dividing into 4 categories
y = to_categorical(y,4)
print(y)
#np.delete(X,1)
#np.delete(X,2)
print(X.shape)
print(y.shape)

[[[[228 172 148]
   [228 172 148]
   [228 172 147]
   ...
   [223 165 147]
   [223 165 147]
   [225 166 149]]

  [[226 170 146]
   [229 173 149]
   [228 172 148]
   ...
   [222 164 146]
   [224 165 148]
   [224 166 148]]

  [[227 171 146]
   [227 171 146]
   [227 171 146]
   ...
   [222 163 146]
   [223 164 147]
   [222 165 144]]

  ...

  [[228 172 147]
   [228 172 148]
   [229 173 148]
   ...
   [219 163 138]
   [219 164 139]
   [218 162 137]]

  [[227 171 147]
   [228 172 148]
   [229 173 149]
   ...
   [220 164 139]
   [219 163 138]
   [219 163 138]]

  [[228 172 147]
   [227 171 147]
   [227 171 146]
   ...
   [220 164 139]
   [220 164 139]
   [219 163 138]]]


 [[[248 188 178]
   [248 187 177]
   [249 188 178]
   ...
   [212 167 163]
   [215 170 165]
   [225 176 171]]

  [[243 187 176]
   [248 187 177]
   [247 186 176]
   ...
   [216 169 164]
   [223 175 170]
   [235 177 173]]

  [[243 187 176]
   [247 187 177]
   [247 187 177]
   ...
   [220 172 168]
   [225 177 173]
   [236 178

In [7]:
#dividing the entire dataset into training data and validation data
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.2,random_state=42)
#dividing the training dataset into training and testing
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train,test_size=0.2,random_state=42)
X_train.shape,y_train.shape,X_valid.shape,y_valid.shape,X_test.shape,y_test.shape
# sum should result in total

((3340, 150, 150, 3),
 (3340, 4),
 (1044, 150, 150, 3),
 (1044, 4),
 (836, 150, 150, 3),
 (836, 4))

In [0]:
batch_size = 64
# as we increase the number of epochs the accuracy will increase
epochs = 20
num_classes = y.shape[1]